# Sampling and Sampling Distributions

A key connection between probability and statistics is the concept of **sampling distributions**.

## Sampling

Before we dig into sampling distributions and statistical theory, we need to understand sampling. We have several pre-packaged commands in R that allow is to quickly generate sample data from a specific distribution. We can also sample rows from a dataframe. The pages in this section will demonstrate how to use these functions together with specific examples.

## Sampling Distributions

A sampling distribution is the set of all samples of the same size drawn from the same population. The statistical theory involved includes:

- Central Limit Theorem (CLT)
- Law of Large Numbers